# Hotel Reservation Dataset - Revisited

---

**Revisiting the Hotels**

Previously used this dataset for a classification project with the goal of modeling the likelihood of a given reservation canceling

<i>New Approach</i>

This time, I'll use the "ReservationStatus" field to perform a multi-class classification model Such a model would be more useful in a practical sense

---

# Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn import set_config
set_config(display = 'Diagram', transform_output = 'pandas')

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 50)

%matplotlib inline

# Import Data

In [2]:
data_path_resort = './data/Resort_Data.xlsx'
data_path_city = './data/City_Data.xlsx'

In [3]:
data_resort = pd.read_excel(data_path_resort)
data_resort['HotelType'] = 'Resort'
data_resort.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelType
0,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort
1,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort
2,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort
3,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort
4,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,Resort


In [4]:
data_city = pd.read_excel(data_path_city)
data_city['HotelType'] = 'City'
data_city.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelType
0,0,6,2015,July,27,1,0,2,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-03,City
1,1,88,2015,July,27,1,0,4,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.50,0,1,Canceled,2015-07-01,City
2,1,65,2015,July,27,1,0,4,1,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,68.00,0,1,Canceled,2015-04-30,City
3,1,92,2015,July,27,1,2,4,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.50,0,2,Canceled,2015-06-23,City
4,1,100,2015,July,27,2,0,2,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.50,0,1,Canceled,2015-04-02,City


In [5]:
data_full = pd.concat([data_resort, data_city])
data_full.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelType
0,0,342,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort
1,0,737,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort
2,0,7,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort
3,0,13,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort
4,0,14,2015,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,Resort


# Feature Exploration

## Numeric Data

In [6]:
data_full.describe(include = ['number', 'datetime'], datetime_is_numeric=True).T

,count,mean,min,25%,50%,75%,max,std
IsCanceled,"119,390.00",0.37,0.00,0.00,0.00,1.00,1.00,0.48
LeadTime,"119,390.00",104.01,0.00,18.00,69.00,160.00,737.00,106.86
ArrivalDateYear,"119,390.00","2,016.16","2,015.00","2,016.00","2,016.00","2,017.00","2,017.00",0.71
ArrivalDateWeekNumber,"119,390.00",27.17,1.00,16.00,28.00,38.00,53.00,13.61
ArrivalDateDayOfMonth,"119,390.00",15.80,1.00,8.00,16.00,23.00,31.00,8.78
StaysInWeekendNights,"119,390.00",0.93,0.00,0.00,1.00,2.00,19.00,1.00
StaysInWeekNights,"119,390.00",2.50,0.00,1.00,2.00,3.00,50.00,1.91
Adults,"119,390.00",1.86,0.00,2.00,2.00,2.00,55.00,0.58
Children,"119,386.00",0.10,0.00,0.00,0.00,0.00,10.00,0.40
Babies,"119,390.00",0.01,0.00,0.00,0.00,0.00,10.00,0.10


In [7]:
data_full.select_dtypes(include = 'number').isna().sum()

IsCanceled                     0
LeadTime                       0
ArrivalDateYear                0
ArrivalDateWeekNumber          0
ArrivalDateDayOfMonth          0
StaysInWeekendNights           0
StaysInWeekNights              0
Adults                         0
Children                       4
Babies                         0
IsRepeatedGuest                0
PreviousCancellations          0
PreviousBookingsNotCanceled    0
BookingChanges                 0
DaysInWaitingList              0
ADR                            0
RequiredCarParkingSpaces       0
TotalOfSpecialRequests         0
dtype: int64

---

**Numeric Feature Review**

> Null or missing values:
> * Children
>
> 
> Features to drop:
> * IsCanceled (too similar to target feature)
>
>
> Treat as ordinal features:
> * ArrivalDateYear
> * ArrivalDateWeekNumber
> * ArrivalDateDayOfMonth
>
>
> Expect outliers in most of the features
> * Determine how to handle later
>   * Eg, Huber regressor can handle them, but other linreg models can't
>
> Features unlikely to be known prior to arrival/guest stay:
> * None, could make argument for all to be known due to reservation booking or calculated from reservation details (eg, number of changes)

---

## Non-Numeric (Strings, DateTime, or Categoricals)

In [8]:
data_full.describe(exclude = ['number', 'datetime'], datetime_is_numeric=True).T

,count,unique,top,freq
ArrivalDateMonth,119390,12,August,13877
Meal,119390,5,BB,92310
Country,118902,177,PRT,48590
MarketSegment,119390,8,Online TA,56477
DistributionChannel,119390,5,TA/TO,97870
ReservedRoomType,119390,10,A,85994
AssignedRoomType,119390,12,A,74053
DepositType,119390,3,No Deposit,104641
Agent,119390,334,9,31961
Company,119390,353,NULL,112593


In [9]:
data_full.select_dtypes(exclude = 'number').isna().sum()

ArrivalDateMonth           0
Meal                       0
Country                  488
MarketSegment              0
DistributionChannel        0
ReservedRoomType           0
AssignedRoomType           0
DepositType                0
Agent                      0
Company                    0
CustomerType               0
ReservationStatus          0
ReservationStatusDate      0
HotelType                  0
dtype: int64

---

**Non-Numeric Feature Review**

> Null or missing values:
>  * Country
>
> 
> Convert to ordinal: ArrivalDateMonth, Meal
>
> Further review needed:
> * MarketSegment vs DistributionChannel
> * ReservedRoomType == AssignedRoomType
> * Company value counts
> * ReservationStatus compared to IsCanceled (possibly too similar to target)
>
> Features unlikely to be known prior to arrival/guest stay:
> * Meal


---

In [10]:
data_full['ReservationStatus'].value_counts(normalize = True, ascending = False).round(2)

Check-Out   0.63
Canceled    0.36
No-Show     0.01
Name: ReservationStatus, dtype: float64

# Initial Data Processing

---

Prior to starting my detailed feature inspection, I need to change a few things:

1 Compare features `IsCanceled` and `ReservationStatus` to determine similarity
2 Convert `ArrivalDateMonth` from month names to month numbers (to treat as ordinal)

---

## Inspect Target Feature

In [11]:
data_full[['IsCanceled', 'ReservationStatus']].head()

,IsCanceled,ReservationStatus
0,0,Check-Out
1,0,Check-Out
2,0,Check-Out
3,0,Check-Out
4,0,Check-Out


In [12]:
data_full[['IsCanceled', 'ReservationStatus']].value_counts(normalize = False, ascending = False)

IsCanceled  ReservationStatus
0           Check-Out            75166
1           Canceled             43017
            No-Show               1207
dtype: int64

In [13]:
data_full[['IsCanceled', 'ReservationStatus']].value_counts(normalize = True, ascending = False).round(2)

IsCanceled  ReservationStatus
0           Check-Out           0.63
1           Canceled            0.36
            No-Show             0.01
dtype: float64

---

ReservationStatus consists of three classes, but the "No-Show" class makes up only 1% of the data

Previously, I considered it to be part of the "Canceled" class, but I want to see how well the model performs with the feature

---

## Inspect NaN/Missing Values

In [14]:
data_full[data_full['Children'].isna()]

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelType
540,1,2,2015,August,32,3,1,0,2,NaN,0,BB,PRT,Undefined,Undefined,0,0,0,B,B,0,No Deposit,NULL,NULL,0,Transient-Party,12.00,0,1,Canceled,2015-08-01,City
607,1,1,2015,August,32,5,0,2,2,NaN,0,BB,PRT,Direct,Undefined,0,0,0,B,B,0,No Deposit,14,NULL,0,Transient-Party,12.00,0,1,Canceled,2015-08-04,City
619,1,1,2015,August,32,5,0,2,3,NaN,0,BB,PRT,Undefined,Undefined,0,0,0,B,B,0,No Deposit,NULL,NULL,0,Transient-Party,18.00,0,2,Canceled,2015-08-04,City
1100,1,8,2015,August,33,13,2,5,2,NaN,0,BB,PRT,Online TA,Undefined,0,0,0,B,B,0,No Deposit,9,NULL,0,Transient-Party,76.50,0,1,Canceled,2015-08-09,City


In [15]:
data_full[data_full['Country'].isna()]

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelType
30,0,118,2015,July,27,1,4,10,1,0.00,0,BB,NaN,Direct,Direct,0,0,0,A,A,2,No Deposit,NULL,NULL,0,Transient,62.00,0,2,Check-Out,2015-07-15,Resort
4127,1,0,2016,February,8,15,0,0,0,0.00,0,SC,NaN,Offline TA/TO,TA/TO,0,0,0,P,P,0,No Deposit,NULL,383,0,Transient,0.00,0,0,Canceled,2016-02-15,Resort
7092,1,8,2016,July,30,21,0,1,1,0.00,0,BB,NaN,Corporate,Corporate,0,0,0,A,A,0,No Deposit,NULL,204,0,Transient,73.00,0,2,Canceled,2016-07-20,Resort
7860,1,39,2016,August,36,30,0,5,2,0.00,0,HB,NaN,Direct,Direct,0,0,0,A,A,0,No Deposit,NULL,NULL,0,Transient,159.00,0,5,Canceled,2016-07-22,Resort
8779,1,0,2016,October,42,13,0,1,1,0.00,0,BB,NaN,Corporate,Corporate,0,0,0,A,A,0,No Deposit,NULL,457,0,Transient,50.00,0,0,Canceled,2016-10-13,Resort
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25848,1,0,2017,April,15,10,0,0,0,0.00,0,SC,NaN,Complementary,Corporate,0,0,0,P,P,0,No Deposit,NULL,279,0,Transient,0.00,0,0,Canceled,2017-04-10,City
25849,1,0,2017,April,15,10,0,0,0,0.00,0,SC,NaN,Complementary,Corporate,0,0,0,P,P,0,No Deposit,NULL,279,0,Transient,0.00,0,0,Canceled,2017-04-10,City
25850,1,0,2017,April,15,10,0,0,0,0.00,0,SC,NaN,Complementary,Corporate,0,0,0,P,P,0,No Deposit,NULL,279,0,Transient,0.00,0,0,Canceled,2017-04-10,City
40770,0,4,2015,November,48,23,1,2,1,0.00,0,BB,NaN,Groups,TA/TO,0,0,0,A,A,0,No Deposit,37,NULL,0,Transient-Party,70.00,0,0,Check-Out,2015-11-26,City


---

With so few missing values, a SimpleImputer can easily handle the inputation of the most frequent values for each feature

---

In [16]:
data_full.loc[:, 'ArrivalDateMonth'].value_counts(normalize = True, ascending = False).round(2)

August      0.12
July        0.11
May         0.10
October     0.09
April       0.09
June        0.09
September   0.09
March       0.08
February    0.07
November    0.06
December    0.06
January     0.05
Name: ArrivalDateMonth, dtype: float64

In [17]:
cond = [data_full['ArrivalDateMonth'] == 'January',
        data_full['ArrivalDateMonth'] == 'February',
        data_full['ArrivalDateMonth'] == 'March',
        data_full['ArrivalDateMonth'] == 'April',
        data_full['ArrivalDateMonth'] == 'May',
        data_full['ArrivalDateMonth'] == 'June',
        data_full['ArrivalDateMonth'] == 'July',
        data_full['ArrivalDateMonth'] == 'August',
        data_full['ArrivalDateMonth'] == 'September',
        data_full['ArrivalDateMonth'] == 'October',
        data_full['ArrivalDateMonth'] == 'November',
        data_full['ArrivalDateMonth'] == 'December']

choice = [1,2,3,4,5,6,7,8,9,10,11,12]

data_full.loc[:, 'ArrivalDateMonth'] = np.select(cond, choice, "Error")

In [18]:
data_full.loc[:, 'ArrivalDateMonth'].value_counts(normalize = True, ascending = False).round(2)

8    0.12
7    0.11
5    0.10
10   0.09
4    0.09
6    0.09
9    0.09
3    0.08
2    0.07
11   0.06
12   0.06
1    0.05
Name: ArrivalDateMonth, dtype: float64

## Drop Unused Columns

In [19]:
df_for_model = data_full.drop(columns = ['IsCanceled', 'ReservationStatusDate', 'ArrivalDateYear']).copy()
df_for_model

,LeadTime,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelType
0,342,7,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,Resort
1,737,7,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,Resort
2,7,7,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,Resort
3,13,7,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,Resort
4,14,7,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,98.00,0,1,Check-Out,Resort
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79325,23,8,35,30,2,5,2,0.00,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394,NULL,0,Transient,96.14,0,0,Check-Out,City
79326,102,8,35,31,2,5,3,0.00,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9,NULL,0,Transient,225.43,0,2,Check-Out,City
79327,34,8,35,31,2,5,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,157.71,0,4,Check-Out,City
79328,109,8,35,31,2,5,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89,NULL,0,Transient,104.40,0,0,Check-Out,City


# Split the Data

In [20]:
## Splitting data into features and target variables.
target= 'ReservationStatus'

X = df_for_model.drop(columns = [target]).copy()
y = df_for_model[target].copy()

In [21]:
## Splitting - stratify to maintain class balance b/t X_train/_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, 
                                                    random_state=42, 
                                                    stratify=y)

# Create Column Transformers

In [22]:
## Specifying numeric columns for preprocessing
num_cols = X_train.select_dtypes('number').columns.to_list()
num_cols.remove('ArrivalDateWeekNumber')
num_cols.remove('ArrivalDateDayOfMonth')
num_cols

['LeadTime',
 'StaysInWeekendNights',
 'StaysInWeekNights',
 'Adults',
 'Children',
 'Babies',
 'IsRepeatedGuest',
 'PreviousCancellations',
 'PreviousBookingsNotCanceled',
 'BookingChanges',
 'DaysInWaitingList',
 'ADR',
 'RequiredCarParkingSpaces',
 'TotalOfSpecialRequests']

In [23]:
## Specifying numeric columns for preprocessing
cat_cols = X_train.select_dtypes(include='object').columns.to_list()
cat_cols.extend(['ArrivalDateWeekNumber','ArrivalDateDayOfMonth'])
cat_cols

['ArrivalDateMonth',
 'Meal',
 'Country',
 'MarketSegment',
 'DistributionChannel',
 'ReservedRoomType',
 'AssignedRoomType',
 'DepositType',
 'Agent',
 'Company',
 'CustomerType',
 'HotelType',
 'ArrivalDateWeekNumber',
 'ArrivalDateDayOfMonth']

In [24]:
X_train['ArrivalDateMonth']

24217     3
7189      7
37339    10
13276     6
14430     2
         ..
15114    10
69550     4
41543     5
42945     1
74856     6
Name: ArrivalDateMonth, Length: 89542, dtype: object

In [25]:
(['ArrivalDateWeekNumber','ArrivalDateDayOfMonth'])

['ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth']

In [ ]:
## Creating ColumnTransformer and sub-transformers for imputation and encoding

### --- Creating column pipelines --- ###

cat_pipe = Pipeline(steps=[('imputer', SimpleImputer(strategy='most frequent'),
                           ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)])

num_pipe = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'),
                            ('scaler', StandardScaler()])

ord_pipe = Pipeline(steps=[('imputer', SimpleImputer(strategy='most frequent'),
                            ('ordenc', OrdinalEncoder()])

### --- Instantiating the ColumnTransformer --- ###
preprocessor = ColumnTransformer(transformers=[('num', num_pipe, num_cols),
                                               ('cat', cat_pipe, cat_cols),
                                               ('ord', ord_pipe, ord_cols)])

preprocessor